In [ ]:
import time

start_time = time.time()

In [ ]:
import cv2
import mediapipe as mp
import json
from tqdm import tqdm

In [ ]:
# Pad naar je video
video_path = r"C:\Users\maxim\OneDrive\01-Opleidingen\03-MAAI\Afstuderen\Data\Video's holland mechanics\machine cover.mp4"  # <-- verander dit
output_json_path = 'handdetectie_resultaten_machine_cover_2.json'

In [ ]:
# Initialiseer MediaPipe Hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.5)

In [ ]:
# Video openen
cap = cv2.VideoCapture(video_path)
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
fps = cap.get(cv2.CAP_PROP_FPS)

results_per_frame = []

# Verwerk elk frame
for frame_idx in tqdm(range(frame_count), desc="Frames verwerken"):
    ret, frame = cap.read()
    if not ret:
        break

    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(frame_rgb)

    time_sec = frame_idx / fps
    frame_data = {
        "frame": frame_idx,
        "time": time_sec,
        "hands": []
    }

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            landmarks = []
            for lm in hand_landmarks.landmark:
                landmarks.append({
                    "x": lm.x,
                    "y": lm.y,
                    "z": lm.z
                })
            frame_data["hands"].append(landmarks)

    results_per_frame.append(frame_data)


In [ ]:
# Opslaan naar JSON-bestand
with open(output_json_path, 'w') as f:
    json.dump(results_per_frame, f, indent=2)

cap.release()
print(f"Resultaten opgeslagen in: {output_json_path}")

In [ ]:
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Totaal uitgevoerde tijd: {elapsed_time:.2f} seconden")

In [ ]:
import cv2
import mediapipe as mp
import json
from tqdm import tqdm

# Pad naar input en output bestanden
video_path = r"C:\Users\maxim\OneDrive\01-Opleidingen\03-MAAI\Afstuderen\Data\Video's holland mechanics\stabilizer pressure control.mp4"
output_json_path = 'handdetectie_resultaten_stabilizer pressure control_2.json'
output_video_path = 'visualisatie_handdetectie.mp4'

# Initialiseer MediaPipe Hands
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.5)

# Open video
cap = cv2.VideoCapture(video_path)
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
fps = cap.get(cv2.CAP_PROP_FPS)

# Initialiseer video writer
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

results_per_frame = []

# Verwerk elk frame
for frame_idx in tqdm(range(frame_count), desc="Frames verwerken"):
    ret, frame = cap.read()
    if not ret:
        break

    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(frame_rgb)

    time_sec = frame_idx / fps
    frame_data = {
        "frame": frame_idx,
        "time": time_sec,
        "hands": []
    }

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            landmarks = []
            for lm in hand_landmarks.landmark:
                landmarks.append({
                    "x": lm.x,
                    "y": lm.y,
                    "z": lm.z
                })
            frame_data["hands"].append(landmarks)

            # Teken landmarks op het frame
            mp_drawing.draw_landmarks(
                frame,
                hand_landmarks,
                mp_hands.HAND_CONNECTIONS
            )

    results_per_frame.append(frame_data)

    # Schrijf het frame met tekeningen naar de output video
    out.write(frame)

# Opslaan als JSON
with open(output_json_path, 'w') as f:
    json.dump(results_per_frame, f, indent=2)

# Opruimen
cap.release()
out.release()
hands.close()
